In [4]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd

# 获取文件路径和文件名
def get_file_details(file, root):
    file_path = os.path.join(root, file)
    return (file_path, file)

# 获取指定目录下所有文件路径和文件名
def get_files_in_directory(directory):
    file_details = []
    futures = []
    
    # 使用 ThreadPoolExecutor 并行处理每个文件
    with ThreadPoolExecutor(max_workers=4) as executor:
        for root, dirs, files in os.walk(directory):
            # 为每个文件提交任务，并记录进度
            for file in files:
                futures.append(executor.submit(get_file_details, file, root))
        
        # 使用 tqdm 显示进度
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing files", unit="file"):
            file_details.append(future.result())
    
    return file_details

# 用并行加速获取文件
def parallel_get_files(directory, num_threads=4):
    # 使用 ThreadPoolExecutor 并行化文件获取
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # 获取文件的路径和文件名
        futures = executor.submit(get_files_in_directory, directory)
        return futures.result()

# 保存文件路径和文件名到 CSV 文件
def save_to_csv(file_details, output_csv):
    # 使用 pandas 处理 CSV
    df = pd.DataFrame(file_details, columns=['path', 'filename'])
    df.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"文件路径和文件名已保存到 {output_csv}")

In [2]:
import os
import pandas as pd
import shutil
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm  # 用于显示进度条

# === 配置部分 ===
csv_path = '/home/jinqiao/mhi/dataset/data/mhi/panda_multi_aes48_coarse_ocr5_flow.csv'  # CSV 文件路径
source_root = '/home/jinqiao/panda/nvme/tmp/heyinan/panda'  # 源文件夹路径
dest_root = '/home/jinqiao/Dataset/panda'  # 目标根路径（CSV 中 path 相对于此）
matched_files_csv = '/home/jinqiao/mhi/matched_files.csv'  # 现有匹配文件的 CSV 文件路径

# === 加载 CSV 文件 ===
df = pd.read_csv(csv_path)
filename_to_relpath = set(df['filename'])  # 使用集合提升查找速度

# === 定义匹配文件的函数 ===
def match_files_in_directory(dirpath, filenames):
    matched_files = []
    rel_dir = os.path.relpath(dirpath, source_root)
    for fname in filenames:
        if fname in filename_to_relpath:
            src = os.path.join(dirpath, fname)
            dst = os.path.join(dest_root, rel_dir, fname)
            matched_files.append((src, dst))
    return matched_files

# === 定义文件移动的函数 ===
def move_file(file_tuple):
    src, dst = file_tuple
    # 创建目标目录（如不存在）
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    # 移动文件
    shutil.move(src, dst)

# === 存储匹配的文件列表 ===
matched_files = []

# === 使用多进程加速匹配 ===
with ProcessPoolExecutor() as executor:
    futures = []
    for dirpath, dirnames, filenames in os.walk(source_root):
        futures.append(executor.submit(match_files_in_directory, dirpath, filenames))

    # 合并匹配的文件
    for future in futures:
        matched_files.extend(future.result())

# === 显示进度条并存储文件 ===
total_files = len(matched_files)

# === 读取现有的 CSV 文件并追加新数据 ===
if os.path.exists(matched_files_csv):
    # 读取现有的 CSV 文件
    existing_df = pd.read_csv(matched_files_csv)
    # 追加新匹配的文件
    new_df = pd.DataFrame(matched_files, columns=["source_path", "destination_path"])
    combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    # 保存到 CSV 文件
    combined_df.to_csv(matched_files_csv, index=False)
else:
    # 如果文件不存在，则创建新的 CSV 文件
    matched_files_df = pd.DataFrame(matched_files, columns=["source_path", "destination_path"])
    matched_files_df.to_csv(matched_files_csv, index=False)

# === 使用进度条显示移动过程 ===
with tqdm(total=total_files, desc="移动文件") as pbar:
    with ProcessPoolExecutor() as executor:
        # 使用 executor.map 并行移动文件，并更新进度条
        for _ in executor.map(move_file, matched_files):
            pbar.update(1)

print("文件移动完成，并且匹配文件已保存。")



移动文件: 100%|██████████| 46701/46701 [00:23<00:00, 2028.39it/s]

文件移动完成，并且匹配文件已保存。


In [ ]:
import pandas as pd
import os

# 读取 CSV 文件
df = pd.read_csv('/home/jinqiao/mhi/matched_files.csv')

# 从 'destination_path' 列提取文件名，并新增到 'filename' 列
df['filename'] = df['destination_path'].apply(lambda x: os.path.basename(x))

# 保存更新后的 DataFrame 到新的 CSV 文件
df.to_csv('/home/jinqiao/matched_files.csv', index=False)

print("文件名列已成功添加到 CSV 文件。")


FileNotFoundError: [Errno 2] No such file or directory: '/home/jinqiao/matched_files.csv'